In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from src.utils.filter_conditions_monthly import filter_conditions_monthly

EUROSTAT_PATH = "src/data/raw/eurostat/latest_data.csv"
BNETZA_PATH = "src/data/raw/germany_household/latest_data.csv"
CBS_PATH = "src/data/raw/CBS_dutch_power.csv"

In [16]:
# 1. Load and preprocess daily demand data
df = pd.read_csv("src/data/processed/daily_demand_all.csv")
df['date'] = pd.to_datetime(df['date'].astype(str).str[:10])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Group by relevant columns and convert to TWh
df = df.groupby(['country', 'type', 'source', 'year', 'month'], as_index=False).agg({'demand': 'sum'})
df['demand'] = df['demand'] / 1000000000

In [17]:
# 2. Load and process Eurostat data
eurostat_df = pd.read_csv(EUROSTAT_PATH)
eurostat_df['date'] = pd.to_datetime(eurostat_df['date'])
eurostat_df['month'] = eurostat_df['date'].dt.month
eurostat_df['year'] = eurostat_df['date'].dt.year
del eurostat_df['date']
df = pd.concat([df, eurostat_df], ignore_index=True)

In [18]:
# 3. Load and process BNetzA data 
bnetza_df = pd.read_csv(BNETZA_PATH)
bnetza_df['date'] = pd.to_datetime(bnetza_df['date'])
bnetza_df['month'] = bnetza_df['date'].dt.month
bnetza_df['year'] = bnetza_df['date'].dt.year
bnetza_append = bnetza_df[['country', 'type', 'source', 'year', 'month', 'demand']]
df = pd.concat([df, bnetza_append], ignore_index=True)

In [19]:
# 4. Load and process CBS data
# we use historic CBS power data (pre July 2022) for power values and converting industry-power to industry
cbs_df = pd.read_csv(CBS_PATH)

#set the month and year columns as datetypes for each
cbs_df['month'] = pd.to_datetime(cbs_df['month'], format="%m").dt.month
cbs_df['year'] = pd.to_datetime(cbs_df['year'], format="%Y").dt.year

#isolate NL and 'industry-power' rows
df_nl_temp = df[(df['country'] == 'NL') & (df['type'] == 'industry')].copy()

#adjust older industry-power rows to industry (by subtracting power)
df_merge = df_nl_temp.merge(cbs_df[['demand','year','month']], on=['year','month'],how='left')
df_merge.replace(np.nan,0,inplace=True)
df_merge['demand'] = df_merge['demand_x'] - df_merge['demand_y']
df_merge['source'] = "entsog, CBS"

df_merge = df_merge[['country', 'type', 'source', 'year', 'month', 'demand']].copy()
cbs_df = cbs_df[['country', 'type', 'source', 'year', 'month', 'demand']].copy()

df = pd.concat([df, df_merge], ignore_index=True)
df = pd.concat([df, cbs_df], ignore_index=True)

In [20]:
# 5. Apply filtering conditions

conditions_df = pd.DataFrame(filter_conditions_monthly, columns=['country', 'type', 'source'])
filtered_df = df.merge(conditions_df, on=['country', 'type', 'source'])

print("\nAfter applying filters:")
display(filtered_df.head())


After applying filters:


,country,type,source,year,month,demand
0,AT,power,energy-charts,2018,12,0.002566
1,AT,power,energy-charts,2019,1,3.147638
2,AT,power,energy-charts,2019,2,2.488796
3,AT,power,energy-charts,2019,3,1.331403
4,AT,power,energy-charts,2019,4,1.055020


In [21]:
# 6. Aggregate data
aggregated_df = (
    filtered_df.groupby(['country', 'type', 'year', 'month'], as_index=False)
    .agg({
        'demand': 'sum',
        'source': lambda x: ', '.join(sorted(set(x))) if len(set(x)) > 1 else x.iloc[0]
    })
)

In [22]:
def track_available_demand(df):
    pivot_df = df.pivot_table(index=["country", "year", "month"], columns="type", values="demand", aggfunc="sum")
    
    records = []
    
    for index, row in pivot_df.iterrows():
        country, year, month = index
        existing_types = set(row.dropna().index)
        
        records.append({
            "country": country,
            "year": year,
            "month": month,
            "available_types": list(existing_types)
        })
    
    tracking_df = pd.DataFrame(records)
    return tracking_df


def adjust_demand(aggregated_df, new_type, operation, required_types_in=None, required_types_not_in=None):
    """
    param: 
    return:
    """
    tracking_available_demand = track_available_demand(aggregated_df)
    
    required_types_in = required_types_in if required_types_in is not None else set()
    required_types_not_in = required_types_not_in if required_types_not_in is not None else set()
    
    valid_entries = tracking_available_demand[
        tracking_available_demand['available_types'].apply(
            lambda x: required_types_in.issubset(set(x)) and required_types_not_in.isdisjoint(set(x))
        )
    ]
    
    new_rows = []
    modified_entries = set()
    
    for _, row in valid_entries.iterrows():
        country, year, month = row['country'], row['year'], row['month']
        subset = aggregated_df[(aggregated_df['country'] == country) &
                               (aggregated_df['year'] == year) &
                               (aggregated_df['month'] == month)]
        
        new_demand = operation(subset)
        if new_demand is not None:
            new_rows.append({
                'country': country,
                'year': year,
                'month': month,
                'type': new_type,
                'demand': new_demand,
                'source': 'calculated'
            })
            modified_entries.add((country, year, month))  # Track modified entries
    
    if new_rows:
        new_df = pd.DataFrame(new_rows)
        aggregated_df = pd.concat([aggregated_df, new_df], ignore_index=True)
    
    return aggregated_df, modified_entries



def industry_str_demand_operation(subset):
    industry_power_demand = subset[subset['type'] == 'industry-power']['demand'].values
    power_demand = subset[subset['type'] == 'power']['demand'].values
    return industry_power_demand[0] - power_demand[0] if len(industry_power_demand) > 0 and len(power_demand) > 0 else None


def industry_sub_demand_operation(subset):
    total_demand = subset[subset['type'] == 'total']['demand'].values
    power_demand = subset[subset['type'] == 'power']['demand'].values
    household_demand = subset[subset['type'] == 'household']['demand'].values
    return total_demand[0] - power_demand[0] - household_demand[0] if len(total_demand) > 0 and len(power_demand) > 0 and len(household_demand) > 0 else None


def total_demand_operation(subset):
    return subset['demand'].sum()


def industry_household_demand_operation(subset):
    total_demand = subset[subset['type'] == 'total']['demand'].values
    power_demand = subset[subset['type'] == 'power']['demand'].values
    return total_demand[0] - power_demand[0] if len(total_demand) > 0 and len(power_demand) > 0 else None

# Apply operations dynamically based on missing data
updated_aggregated_df, modified_industry_str = adjust_demand(
    aggregated_df, 'industry', industry_sub_demand_operation, 
    {'total', 'power', 'household'}, {'industry'}
)

updated_aggregated_df, modified_industry_str = adjust_demand(
    updated_aggregated_df, 'industry', industry_str_demand_operation, 
    {'industry-power', 'power', 'household'}, {'industry'}
)

# Remove 'industry-power' only for modified entries
updated_aggregated_df = updated_aggregated_df[
    ~((updated_aggregated_df['type'] == 'industry-power') & 
      (updated_aggregated_df[['country', 'year', 'month']].apply(tuple, axis=1).isin(modified_industry_str))
    )
].reset_index(drop=True)

updated_aggregated_df, _ = adjust_demand(
    updated_aggregated_df, 'total', total_demand_operation, None, {'total'}
)


# Remove 'industry' only for modified entries
final_aggregated_df, modified_industry_household = adjust_demand(
    updated_aggregated_df, 'industry-household', industry_household_demand_operation, 
    {'power', 'total'}, {'industry', 'household'}
)


# Recalculate tracking_available_demand
tracking_available_demand = track_available_demand(final_aggregated_df)



In [23]:
# Remove unwanted countries
final_aggregated_df = final_aggregated_df[~final_aggregated_df['country'].isin(["EU27_2020", "CY", "MT"])]

def compute_country_contributions(group):
    total_demand = group['demand'].sum()  # Compute total demand for the group
    
    contributions = {
        country: round(float(group[group['country'] == country]['demand'].sum() / total_demand) * 100, 10)  
        for country in group['country'].unique()
    }
    
    return pd.Series({'country_contributions': contributions, 'demand': total_demand})  # Rename total_demand to demand

# Group by type, year, and month and compute demand + country-wise contributions
df_eu = final_aggregated_df[final_aggregated_df['country'] != "UK"].copy()
df_pivot = df_eu.groupby(['type', 'year', 'month']).apply(compute_country_contributions).reset_index()

# Assign 'EU' as the country
df_pivot['country'] = 'EU'

# Add source column
df_pivot['source'] = 'calculated'

# Convert country contributions to a properly formatted dictionary with float values
df_pivot['country_contributions'] = df_pivot['country_contributions'].apply(lambda x: {k: float(v) for k, v in x.items()})

# Concatenate with the original dataset
final_aggregated_df = pd.concat([final_aggregated_df, df_pivot], ignore_index=True)

/var/folders/65/v55t85jx0k799mrx8ctk7yvr0000gn/T/ipykernel_6406/551365456.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_pivot = df_eu.groupby(['type', 'year', 'month']).apply(compute_country_contributions).reset_index()


In [24]:
final_aggregated_df.tail()

,country,type,year,month,demand,source,country_contributions
7421,EU,total,2024,11,379.517485,calculated,"{'AT': 2.7014535392, 'BG': 0.8001143029, 'CZ':..."
7422,EU,total,2024,12,434.065738,calculated,"{'AT': 2.4673723, 'BG': 0.7090580283, 'CZ': 2...."
7423,EU,total,2025,1,442.426496,calculated,"{'AT': 2.6645924109, 'BG': 0.6967183462, 'CZ':..."
7424,EU,total,2025,2,388.652074,calculated,"{'AT': 2.7390915677, 'BG': 0.80216928, 'DE': 2..."
7425,EU,total,2025,3,17.641120,calculated,"{'AT': 1.5604051423, 'BG': 1.141625828, 'DE': ..."


In [25]:
# 9. Final processing and export
final_aggregated_df = final_aggregated_df[final_aggregated_df['year'] >= 2019]
final_aggregated_df['demand'] = final_aggregated_df['demand'].round(2)

# Filter until March 2025
final_aggregated_df = final_aggregated_df[
    ((final_aggregated_df['year'] < 2025) |
     ((final_aggregated_df['year'] == 2025) & (final_aggregated_df['month'] < 1)))
]


In [26]:
# Save the results
final_aggregated_df.to_csv("src/data/analyzed/monthly_demand_clean.csv", index=False)
final_aggregated_df.to_json("src/data/analyzed/monthly_demand_clean.json", orient='records', indent=4)
with pd.ExcelWriter("src/data/analyzed/monthly_demand_clean.xlsx", engine='openpyxl') as writer:
    final_aggregated_df.to_excel(writer, sheet_name="Aggregated Data", index=False)

In [27]:
#10. compare totals with eurostat
df_check = final_aggregated_df[['country','demand','type','source','month', 'year']].copy()
df_check = df_check[df_check['type'] == 'total'].copy()
df_check['source'] = 'bruegel'

eurostat_check = eurostat_df.copy()
eurostat_check.loc[eurostat_check['country'] == 'EU27_2020', 'country'] = 'EU'

df_check = pd.concat([df_check, eurostat_check])
df_check = df_check[df_check['year'] > 2018].copy()
df_pivot = df_check.pivot_table(index=['country', 'year', 'month'], columns='source', values='demand')

def plot_and_save_countries(df, output_dir="src/figures/eurostat_totals"):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Loop over each country in the DataFrame index
    for country in df.index.get_level_values(0).unique():
        country_df = df.loc[country]  # Select the country
        country_df = country_df.reset_index()  # Make 'year' and 'month' columns

        # Create a combined 'time' column
        country_df['time'] = country_df['year'].astype(str) + '-' + country_df['month'].astype(str)
        
        # Plot
        plt.figure(figsize=(10, 5))
        plt.plot(country_df['time'], country_df['bruegel'], label='Bruegel', marker='o')
        plt.plot(country_df['time'], country_df['eurostat'], label='Eurostat', marker='s')
        
        # Formatting
        plt.xlabel('')
        plt.ylabel('TWh')
        plt.title(f'{country}')

        # Show every 6th tick label
        tick_positions = range(0, len(country_df), 6)  # Every 6th index
        plt.xticks(tick_positions, country_df['time'].iloc[tick_positions], rotation=45)
        
        plt.legend()
        plt.grid(True)
        
        # Save the figure
        plt.savefig(os.path.join(output_dir, f"{country}.png"), dpi=300, bbox_inches='tight')
        plt.close()  # Close the figure to free memory
    
    print(f"Plots saved to {output_dir}")

# Run the function
plot_and_save_countries(df_pivot)

Plots saved to src/figures/eurostat_totals


In [14]:
#### A CHECK THAT EU NUMBERS MAKE SENSE

df_check_eu = final_aggregated_df[(final_aggregated_df['country'] == 'EU')].pivot_table(index=['year','month'],columns='type',values='demand')
df_check_eu['total_calculated'] = df_check_eu[['household','industry','industry-household','power']].sum(axis=1)
df_check_eu

type        household  industry  industry-household  power   total  \
year month                                                           
2019 1         267.86     99.82               75.46  83.36  526.50   
     2         195.83     86.41               58.12  63.36  403.72   
     3         164.99     88.39               52.11  55.49  360.99   
     4         108.00     79.94               41.70  51.12  280.77   
     5          85.55     80.51               37.41  49.85  253.33   
...               ...       ...                 ...    ...     ...   
2024 8          30.10     53.33               24.28  46.38  154.08   
     9          44.30     60.84               28.51  43.72  177.38   
     10         78.99     66.04               39.47  45.72  230.22   
     11        152.49     72.77               54.33  70.49  350.08   
     12        197.96     75.05               59.49  66.45  398.95   

type        total_calculated  
year month                    
2019 1                526.50  
     2                403.72  
     3                360.98  
     4                280.76  
     5                253.32  
...                      ...  
2024 8                154.09  
     9                177.37  
     10               230.22  
     11               350.08  
     12               398.95  

[72 rows x 6 columns]

In [28]:
df_pivot = final_aggregated_df[final_aggregated_df['type'] == 'total'].pivot_table(
    index='country', 
    columns='year', 
    values='demand',
    aggfunc='sum')

df_pivot['AVG'] = df_pivot[[2019,2020,2021]].mean(axis=1)
df_pivot['2022-delta'] = df_pivot[2022] - df_pivot['AVG']
df_pivot['2023-delta'] = df_pivot[2023] - df_pivot['AVG']
df_pivot['2024-delta'] = df_pivot[2024] - df_pivot['AVG']
df_pivot['avg-delta'] = (df_pivot['2022-delta'] + df_pivot['2023-delta'] + df_pivot['2024-delta'])/3

# Add sum row excluding 'EU'
sum_row = df_pivot[df_pivot.index != 'EU'].sum()
df_pivot.loc['TOTAL'] = sum_row

#calculate percentage changes against AVG column
df_pivot['2022-pct-change'] = df_pivot['2022-delta'] / df_pivot['AVG']
df_pivot['2023-pct-change'] = df_pivot['2023-delta'] / df_pivot['AVG']
df_pivot['2024-pct-change'] = df_pivot['2024-delta'] / df_pivot['AVG']
df_pivot['avg-pct-change'] = df_pivot['avg-delta'] / df_pivot['AVG']

df_pivot

year,2019,2020,2021,2022,2023,2024,AVG,2022-delta,2023-delta,2024-delta,avg-delta,2022-pct-change,2023-pct-change,2024-pct-change,avg-pct-change
country,,,,,,,,,,,,,,,
AT,92.53,90.62,96.23,85.15,75.07,74.47,93.126667,-7.976667,-18.056667,-18.656667,-14.896667,-0.085654,-0.193894,-0.200336,-0.159961
BE,192.06,190.65,189.99,161.29,152.07,149.05,190.900000,-29.610000,-38.830000,-41.850000,-36.763333,-0.155107,-0.203405,-0.219225,-0.192579
BG,30.45,31.31,35.33,28.17,26.27,27.85,32.363333,-4.193333,-6.093333,-4.513333,-4.933333,-0.129571,-0.188279,-0.139458,-0.152436
CZ,92.52,94.04,100.76,84.49,74.42,74.22,95.773333,-11.283333,-21.353333,-21.553333,-18.063333,-0.117813,-0.222957,-0.225045,-0.188605
DE,972.18,966.66,1034.62,869.76,820.18,838.31,991.153333,-121.393333,-170.973333,-152.843333,-148.403333,-0.122477,-0.172499,-0.154208,-0.149728
DK,28.94,26.20,27.73,20.81,20.77,21.23,27.623333,-6.813333,-6.853333,-6.393333,-6.686667,-0.246651,-0.248099,-0.231447,-0.242066
EE,4.76,4.43,5.06,3.76,3.42,3.68,4.750000,-0.990000,-1.330000,-1.070000,-1.130000,-0.208421,-0.280000,-0.225263,-0.237895
ES,398.26,359.65,378.29,363.47,325.56,311.99,378.733333,-15.263333,-53.173333,-66.743333,-45.060000,-0.040301,-0.140398,-0.176228,-0.118976
EU,4135.82,4051.81,4263.09,3664.25,3381.35,3404.95,4150.240000,-485.990000,-768.890000,-745.290000,-666.723333,-0.117099,-0.185264,-0.179578,-0.160647
